In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
def process_csv():
    # import the csv file
    df = pd.read_csv('ISIC_2019_Training_GroundTruth_v2.csv')